#RUN FROM HERE

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Load the dataset (assuming the path is correctly set)
data_file_path = '/content/drive/MyDrive/GuidingRobot/LIDAR_DATA/LIDAR_DATA_UNZIPPED/lidar_like_data.csv'
df = pd.read_csv(data_file_path)

# Preprocess the DataFrame
df = df.drop(columns=['file_name'])

In [33]:



# Calculate mean squared distance to goal for each row
df['mean_sq_distance'] = (df['P1_goal_dist_x']**2 + df['P1_goal_dist_y']**2) / 2
num_rows_less_than_200 = df[df['mean_sq_distance'] < 100].shape[0]
print("Number of rows with mean_sq_distance less than 200:", num_rows_less_than_200)

# Initially mark all rows as part of an invalid sequence
df['valid_sequence'] = False

for i in reversed(range(len(df))):
    if df.iloc[i]['final_row_flag'] == 1:
        if df.iloc[i]['mean_sq_distance'] < 100:
            # If the sequence ending at row i is successful, mark it and any preceding rows as valid
            if last_final_flag_index == -1:
                last_final_flag_index = i  # For the first successful end sequence
            else:
                for j in range(i, last_final_flag_index + 1):
                    df.at[j, 'valid_sequence'] = True
                    print(f"Row {j} Mean Squared Distance: {df.at[j, 'mean_sq_distance']}")
            last_final_flag_index = i  # Update the last index where final_row_flag was 1
        else:
            # Reset the tracker if the end of the sequence is not successful
            last_final_flag_index = i

df_filtered = df[df['valid_sequence']]
df_filtered = df_filtered.drop(columns=['mean_sq_distance', 'valid_sequence'])


print(df_filtered.columns)



Number of rows with mean_sq_distance less than 200: 1598
Row 21982 Mean Squared Distance: 74.5
Row 21983 Mean Squared Distance: 8448.5
Row 21984 Mean Squared Distance: 8320.0
Row 21985 Mean Squared Distance: 8192.5
Row 21986 Mean Squared Distance: 8066.0
Row 21987 Mean Squared Distance: 7940.5
Row 21988 Mean Squared Distance: 7816.0
Row 21989 Mean Squared Distance: 7692.5
Row 21990 Mean Squared Distance: 7570.0
Row 21991 Mean Squared Distance: 7448.5
Row 21992 Mean Squared Distance: 7328.0
Row 21993 Mean Squared Distance: 7208.5
Row 21994 Mean Squared Distance: 7090.0
Row 21995 Mean Squared Distance: 6972.5
Row 21996 Mean Squared Distance: 6856.0
Row 21997 Mean Squared Distance: 6740.5
Row 21998 Mean Squared Distance: 6626.0
Row 21999 Mean Squared Distance: 6512.5
Row 22000 Mean Squared Distance: 6400.0
Row 22001 Mean Squared Distance: 6288.5
Row 22002 Mean Squared Distance: 6178.0
Row 22003 Mean Squared Distance: 6068.5
Row 22004 Mean Squared Distance: 5960.0
Row 22005 Mean Squared Di

In [34]:
final_state_flag_counts = df_filtered['final_row_flag'].value_counts()
print(final_state_flag_counts)


0.0    1514
1.0       4
Name: final_row_flag, dtype: int64


In [35]:
# Using .shape to get the number of rows and columns
num_rows, num_columns = df_filtered.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

# Using len() to get the number of rows directly
num_rows = len(df_filtered)
print("Number of rows:", num_rows)


Number of rows: 1518
Number of columns: 1084
Number of rows: 1518


In [36]:
# Assuming P1_Direction is the target and all other columns are features
X = df_filtered.drop(['P1_action'], axis=1)
y = df_filtered['P1_action']


##split into train test val

 60% train, 20% validation, and 20% test split

In [37]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2



##convert df to numpy array then tensor

In [38]:

import torch
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
X_val_tensor = torch.tensor(X_val.to_numpy(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)

y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long)
y_val_tensor = torch.tensor(y_val.to_numpy(), dtype=torch.long)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)


##Create Data Loader

In [39]:
# Create TensorDatasets
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
batch_size = 64  # You can adjust the batch size

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


##create model

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


import torch.nn as nn
import torch.nn.functional as F

class BCAgent(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BCAgent, self).__init__()
        # Use the input_dim and output_dim arguments to adjust layer dimensions
        self.fc1 = nn.Linear(input_dim, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.dp1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.dp2 = nn.Dropout(0.2)

        self.fc3 = nn.Linear(512, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.dp3 = nn.Dropout(0.2)

        self.fc4 = nn.Linear(128, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.dp4 = nn.Dropout(0.2)

        self.fc5 = nn.Linear(64, 32)
        self.bn5 = nn.BatchNorm1d(32)
        self.dp5 = nn.Dropout(0.2)

        self.fc6 = nn.Linear(32, output_dim)  # Use output_dim for the output layer

    def forward(self, x):
        x = F.relu(self.dp1(self.bn1(self.fc1(x))))
        x = F.relu(self.dp2(self.bn2(self.fc2(x))))
        x = F.relu(self.dp3(self.bn3(self.fc3(x))))
        x = F.relu(self.dp4(self.bn4(self.fc4(x))))
        x = F.relu(self.dp5(self.bn5(self.fc5(x))))
        x = self.fc6(x)
        return x




In [ ]:
''''
**OLD**

model = Sequential([
    Dense(1024, activation='relu', input_shape=(26,), ),
    Dense(512, activation='relu', ),
    Dense(128, activation='relu', ),
    Dense(64, activation='relu', ),
    Dense(32, activation='relu', ),
    # Update the Dense layer for output according to your classification/regression needs
    Dense(5, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
'''

In [ ]:
'''
model = Sequential([
    Dense(512, activation='relu', input_shape=(26,), ),

    Dense(128, activation='relu', ),

    Dense(32, activation='relu', ),
    # Update the Dense layer for output according to your classification/regression needs
    Dense(5, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
##RUN ON WAND B NAME-SOFT MEDOW
'''

In [ ]:

''''
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

# Assuming X_train.shape[1] is 126, replace it with 126 directly if you're not sure
model = Sequential([
    Dense(1024, activation='relu', input_shape=(26,), ),
    Dense(512, activation='relu', ),
    Dense(256, activation='relu', ),
    Dense(128, activation='relu', ),
    Dense(64, activation='relu', ),
    Dense(32, activation='relu', ),
    # Update the Dense layer for output according to your classification/regression needs
    Dense(5, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
###increasing depth deterioraes performance

##RUN ON WAND B NAME-GIDDY BUTTERFLY

''''

##train

In [25]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00


In [26]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

In [27]:
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbEvalCallback


wandb.login()



<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [28]:
import wandb
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import torch

# EarlyStopping class to stop the training when a monitored metric has stopped improving.
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = np.Inf
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                print('Early stopping')
                self.early_stop = True


In [41]:

# Initialize WandB
wandb.init(project='GRProject Pytorch-LIDAR-CHANGEREP3', config={
    "learning_rate": 0.001,
    "epochs": 20,
    "batch_size": 64,
})
config = wandb.config
input_dim = X.shape[1]  # Number of columns in the DataFrame

model = BCAgent(input_dim=input_dim, output_dim=5)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

early_stopping = EarlyStopping(patience=10, min_delta=0.001)

for epoch in range(config.epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / total_predictions
    epoch_accuracy = (correct_predictions / total_predictions) * 100

    wandb.log({"Train_loss": epoch_loss, "Train_accuracy": epoch_accuracy})

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct_val_predictions = 0
    total_val_predictions = 0

    with torch.no_grad():  # No need to track gradients during validation
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)  # Accumulate the batch loss

            _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability
            total_val_predictions += labels.size(0)
            correct_val_predictions += (predicted == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_accuracy = (correct_val_predictions / total_val_predictions) * 100

    # Log validation loss and accuracy to WandB
    wandb.log({"val_loss": val_loss, "val_accuracy": val_accuracy})
    # Call early stopping
    #early_stopping(_loss)
    #if early_stopping.early_stop:
        #print("Stopped early at epoch", epoch+1)
        #break
wandb.finish()


Train_accuracy,▁▅▅▆▆▇▇▇▇▇▇█████████
Train_loss,█▆▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▂▄▅▅▅▅█▇▇▇▇▅▇▆██▆█
val_loss,█▅▄▄▄▃▂▃▂▂▂▂▂▃▁▂▁▁▂▁
Train_accuracy,91.50391
Train_loss,0.2501
val_accuracy,88.0117
val_loss,0.32827


##Inference

In [42]:
model_save_path = "/content/drive/MyDrive/GuidingRobot/model_checkpoints_relative/model_checkpoint_LIDAR_succesfultrials_1.pth"  # Define your save path here
wandb.init()
torch.save(model.state_dict(), model_save_path)
wandb.save(model_save_path)  # Optionally log the model file to WandB

wandb.finish()

In [43]:
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct_predictions = 0
total_predictions = 0

with torch.no_grad():  # Inference mode, no need to compute gradients
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)  # Accumulate the batch loss

        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

test_loss /= len(test_loader.dataset)
test_accuracy = (correct_predictions / total_predictions) * 100
print("test loss",test_loss)
print("test acc",test_accuracy)



test loss 0.3547324028454329
test acc 88.1578947368421
